In [1]:
import os
import getdata
import sortdata
import plotting 
import numpy as np
import unique_objects_obs

#create results directory inside output folder and clear it at the beginning
dir = os.path.join("output", "results_observations_magnitude_cut")
os.makedirs(dir, exist_ok=True)
getdata.clear_directory(dir)

#files where the uncorrelated observation data is stored
uncorr_obs_2002 = "ogs200208_12_det.ele_ucorr"
uncorr_obs_2003 = "ogs200301_12_det.ele_ucorr"
uncorr_obs_2004 = "ogs200401_12_det.ele_ucorr"
uncorr_obs_2005 = "ogs200501_12_det.ele_ucorr"

uncorr_obs_2006 = "ogs200601_12_det.ele_ucorr"
uncorr_obs_2007 = "ogs200701_12_det.ele_ucorr"
uncorr_obs_2008 = "ogs200801_12_det.ele_ucorr"
uncorr_obs_2009 = "ogs200901_12_det.ele_ucorr"

uncorr_obs_2010 = "ogs201001_12_det.ele_ucorr"
uncorr_obs_2011 = "ogs201101_12_det.ele_ucorr"
uncorr_obs_2012 = "ogs201201_12_det.ele_ucorr"
uncorr_obs_2013 = "ogs201301_12_det.ele_ucorr"

uncorr_obs_2014 = "ogs201401_12_det.ele_ucorr"
uncorr_obs_2015 = "ogs201501_12_det.ele_ucorr"
uncorr_obs_2016 = "ogs201601_12_det.ele_ucorr"
uncorr_obs_2017 = "ogs201701_12_det.ele_ucorr"

uncorr_obs_2019 = "ogs201901_12_det.ele_ucorr"
uncorr_obs_2020 = "ogs202001_12_det.ele_ucorr"
uncorr_obs_2021 = "ogs202101_12_det.ele_ucorr"
uncorr_obs_2022 = "ogs202201_12_det.ele_ucorr"
uncorr_obs_2023 = "ogs202301_12_det.ele_ucorr"

#groups of years (how we want the data grouped in the plots)
year_list = [
    ("2002 - 2005", [uncorr_obs_2002, uncorr_obs_2003, uncorr_obs_2004, uncorr_obs_2005],),
    ("2006 - 2009", [uncorr_obs_2006, uncorr_obs_2007, uncorr_obs_2008, uncorr_obs_2009]),
    ("2010 - 2013", [uncorr_obs_2010, uncorr_obs_2011, uncorr_obs_2012, uncorr_obs_2013]),
    ("2014 - 2017", [uncorr_obs_2014, uncorr_obs_2015, uncorr_obs_2016, uncorr_obs_2017]),
    ("2019 - 2023", [uncorr_obs_2019, uncorr_obs_2020, uncorr_obs_2021, uncorr_obs_2022, uncorr_obs_2023]),
]

#initialize number of objects per year group and orbit type
num_0205_GEO, num_0205_GTO, num_0205_fol = 0, 0, 0
num_0609_GEO, num_0609_GTO, num_0609_fol = 0, 0, 0
num_1013_GEO, num_1013_GTO, num_1013_fol = 0, 0, 0
num_1417_GEO, num_1417_GTO, num_1417_fol = 0, 0, 0
num_1923_GEO, num_1923_GTO, num_1923_fol = 0, 0, 0

for years, obs_files in year_list: #iterate over every (year group, obs file list) in year_list
    number_years = []
    
    for i, obs_file in enumerate(obs_files):
        obs_file = os.path.join("input", obs_file)
        obs_files[i] = obs_file
        year = obs_file[9:13] # Extract the year from the filename 
        number_years.append(year)
    
    print(obs_files) #serves as a progess bar during code execution
    data = getdata.array_extender_obs(obs_files) #array extender is adapted for the format of the obs files

    #sortdata: for apogee, inclination, magnitudes
    semi_major = data[9]
    ecc = data[10]
    inc = data[11]
    mag_u = data[8]
    raan = data[12]
    semi_major = [float(a) for a in semi_major]
    ecc = [float(e) for e in ecc]
    data = sortdata.sort_for_apogee(semi_major, ecc , inc, semi_major, raan, inc, mag_u, ecc)
    inc = data[0]
    semi_major = data[1]
    raan = data[2]
    inc = data[3]
    mag_u = data[4]
    inc = [float(i) for i in inc]
    ecc = data[5]
    data = sortdata.sort_for_inclination(inc, 22, semi_major, raan, inc, mag_u, ecc)
    semi_major = data[0]
    raan = data[1]
    inc = data[2]
    magnitudes = data[3]
    ecc = data[4]
    raan = [float(w) for w in raan]
    magnitudes = [float(m) for m in magnitudes]
    min_mag = 14.5
    
    data = sortdata.sort_for_magnitudes(magnitudes, min_mag, raan, inc, magnitudes, ecc, semi_major)
    raan = data[0]
    inc = data[1]
    magnitudes = data[2]
    ecc = data[3]
    semi_major = data[4]
        
    plotting.i_omega_separate(raan, inc, years, f"Observed detections {years}", years, " ", dir)#i omega plots (all orbit types mixed, for both crossings and detections)
    plotting.i_omega_with_eccentricity(raan, inc, ecc, f"Observed detections {years}", years, dir)#i omega plot where the objects are colored according to their eccentricity
    plotting.i_omega_with_sem_maj(raan, inc, semi_major, f"Observed detections {years}", years, dir)#i omega plot where the objects are colored according to their semi major axis
    plotting.i_omega_with_magnitude(raan, inc, magnitudes, f"Observed detections {years}", years, dir)#i omega plot where the objects are colored according to their semi major axis
    plotting.magnitude_plot(magnitudes, 1923, f"Observed detections {years} ", " ", dir)

#from here on: code to make i omega plots using different colors for different orbit types
    GEO_data = []
    GTO_data = []
    fol_data = []
    path = os.path.join("input", "sorted_observation_files")
    
    
    for year in number_years: 
        GEO, GTO, fol = getdata.extract_separated_obs_files(year, path) #uncorrelated data gets sorted into orbit types
        GEO = np.array(GEO)
        GTO = np.array(GTO)
        fol = np.array(fol)
        
        GEO_data.append(GEO)
        GTO_data.append(GTO)
        fol_data.append(fol)
        
    GEO_data = np.hstack(GEO_data)
    GTO_data = np.hstack(GTO_data)
    fol_data = np.hstack(fol_data)
    
    #comment this out for fast testing of the remaining code
    #define tolerances for the unique algorithm
    tolerances = [0,0,np.Inf, np.Inf, np.Inf, 5000, 0.1, 0.1, 0.1, 0.1, 1, np.Inf, 5000, 0.1, 0.1, 0.1, 0.1, 1, np.Inf, np.Inf, np.Inf, np.Inf, np.Inf, np.Inf, np.Inf, 1, np.Inf, np.Inf, np.Inf, 0]
    data_for_unique = np.hstack([GEO_data, GTO_data, fol_data])
    data_for_unique = sortdata.data_sorter(data_for_unique, 9, 10, 11, 8 )
    uniques = unique_objects_obs.process_obs_space_objects_from_array(data_for_unique.T, tolerances)
    total_objects = len(data_for_unique[1])
    print(f"Unique / total for {years[0]}-{years[-1]}: {len(uniques) / total_objects:.3f}")
        
    #convert everything to float    
    GEO_data_nod = np.array([float(x) for x in GEO_data[12]], dtype=float) #convert all data (str) to float
    GTO_data_nod = np.array([float(x) for x in GTO_data[12]], dtype=float)
    fol_data_nod = np.array([float(x) for x in fol_data[12]], dtype=float)

    GEO_data_inc = np.array([float(x) for x in GEO_data[11]], dtype=float)
    GTO_data_inc = np.array([float(x) for x in GTO_data[11]], dtype=float)
    fol_data_inc = np.array([float(x) for x in fol_data[11]], dtype=float)
    
    GEO_data_ecc = np.array([float(x) for x in GEO_data[10]], dtype=float)
    GTO_data_ecc = np.array([float(x) for x in GTO_data[10]], dtype=float)
    fol_data_ecc = np.array([float(x) for x in fol_data[10]], dtype=float)
    
    GEO_data_semi_major = np.array([float(x) for x in GEO_data[9]], dtype=float)
    GTO_data_semi_major = np.array([float(x) for x in GTO_data[9]], dtype=float)
    fol_data_semi_major = np.array([float(x) for x in fol_data[9]], dtype=float)
    
    GEO_data_mag = np.array([float(x) for x in GEO_data[8]], dtype=float)
    GTO_data_mag = np.array([float(x) for x in GTO_data[8]], dtype=float)
    fol_data_mag = np.array([float(x) for x in fol_data[8]], dtype=float)
    
    #sorting: for apogee, inclination, magnitudes
    GEO_data_nod, GEO_data_inc, GEO_data_mag = sortdata.sort_for_apogee(GEO_data_semi_major, GEO_data_ecc, GEO_data_nod, GEO_data_inc, GEO_data_mag)
    GTO_data_nod, GTO_data_inc, GTO_data_mag = sortdata.sort_for_apogee(GTO_data_semi_major, GTO_data_ecc, GTO_data_nod, GTO_data_inc, GTO_data_mag)
    fol_data_nod, fol_data_inc, fol_data_mag = sortdata.sort_for_apogee(fol_data_semi_major, fol_data_ecc, fol_data_nod, fol_data_inc, fol_data_mag)
    
    max_inc = 22.0
    GEO_data_nod, GEO_data_mag = sortdata.sort_for_inclination(GEO_data_inc, max_inc, GEO_data_nod, GEO_data_mag)
    GTO_data_nod, GTO_data_mag = sortdata.sort_for_inclination(GTO_data_inc, max_inc, GTO_data_nod, GTO_data_mag)
    fol_data_nod, fol_data_mag = sortdata.sort_for_inclination(fol_data_inc, max_inc, fol_data_nod, fol_data_mag)
    
    #set all negative inclinations to zero
    GEO_data_inc = [max(0, i) for i in GEO_data_inc if i < max_inc]
    GTO_data_inc = [max(0, i) for i in GTO_data_inc if i < max_inc]
    fol_data_inc = [max(0, i) for i in fol_data_inc if i < max_inc]
    
    min_mag = 14.5
    GEO_data_nod, GEO_data_inc = sortdata.sort_for_magnitudes(GEO_data_mag, min_mag, GEO_data_nod, GEO_data_inc)
    GTO_data_nod, GTO_data_inc = sortdata.sort_for_magnitudes(GTO_data_mag, min_mag, GTO_data_nod, GTO_data_inc)
    fol_data_nod, fol_data_inc = sortdata.sort_for_magnitudes(fol_data_mag, min_mag, fol_data_nod, fol_data_inc)

    #i omega plot where different orbit types are different colors
    plotting.i_omega_all_orbits(np.array(GEO_data_nod), np.array(GTO_data_nod), np.array(fol_data_nod), np.array(GEO_data_inc), np.array(GTO_data_inc), np.array(fol_data_inc), f"Observed detections {years}", years, dir)
    
    #get number of objects in different orbit types
    num_GEO = len(GEO_data_nod)
    num_GTO = len(GTO_data_nod)
    num_fol = len(fol_data_nod)

    #use different variable names for different years
    if int(years[0:4]) == 2002:
        num_0205_GEO, num_0205_GTO, num_0205_fol = num_GEO, num_GTO, num_fol
    elif int(years[0:4]) == 2006:
        num_0609_GEO, num_0609_GTO, num_0609_fol = num_GEO, num_GTO, num_fol
    elif int(years[0:4]) == 2010:
        num_1013_GEO, num_1013_GTO, num_1013_fol = num_GEO, num_GTO, num_fol
    elif int(years[0:4]) == 2014:
        num_1417_GEO, num_1417_GTO, num_1417_fol = num_GEO, num_GTO, num_fol
    elif int(years[0:4]) == 2019:
        num_1923_GEO, num_1923_GTO, num_1923_fol = num_GEO, num_GTO, num_fol
        
print(f"Number of detections 2002 - 2005 in GEO/GTO/fol: {num_0205_GEO}, {num_0205_GTO}, {num_0205_fol}")
print(f"Number of detections 2006 - 2009 in GEO/GTO/fol: {num_0609_GEO}, {num_0609_GTO}, {num_0609_fol}")
print(f"Number of detections 2010 - 2013 in GEO/GTO/fol: {num_1013_GEO}, {num_1013_GTO}, {num_1013_fol}")
print(f"Number of detections 2014 - 2017 in GEO/GTO/fol: {num_1417_GEO}, {num_1417_GTO}, {num_1417_fol}")
print(f"Number of detections 2019 - 2023 in GEO/GTO/fol: {num_1923_GEO}, {num_1923_GTO}, {num_1923_fol}")

#total across all years per orbit type
total_GEO = num_0205_GEO + num_0609_GEO + num_1013_GEO + num_1417_GEO + num_1923_GEO
total_GTO = num_0205_GTO + num_0609_GTO + num_1013_GTO + num_1417_GTO + num_1923_GTO
total_fol = num_0205_fol + num_0609_fol + num_1013_fol + num_1417_fol + num_1923_fol

print(f"Total number of detections in GEO (all years): {total_GEO}")
print(f"Total number of detections in GTO (all years): {total_GTO}")
print(f"Total number of detections in fol (all years): {total_fol}")

#total per year group all orbit types
total_0205 = num_0205_GEO + num_0205_GTO + num_0205_fol
total_0609 = num_0609_GEO + num_0609_GTO + num_0609_fol
total_1013 = num_1013_GEO + num_1013_GTO + num_1013_fol
total_1417 = num_1417_GEO + num_1417_GTO + num_1417_fol 
total_1923 = num_1923_GEO + num_1923_GTO + num_1923_fol

['input\\ogs200208_12_det.ele_ucorr', 'input\\ogs200301_12_det.ele_ucorr', 'input\\ogs200401_12_det.ele_ucorr', 'input\\ogs200501_12_det.ele_ucorr']
4696
4690
Unique / total for 2-5: 0.999
['input\\ogs200601_12_det.ele_ucorr', 'input\\ogs200701_12_det.ele_ucorr', 'input\\ogs200801_12_det.ele_ucorr', 'input\\ogs200901_12_det.ele_ucorr']
4307
4303
Unique / total for 2-9: 0.999
['input\\ogs201001_12_det.ele_ucorr', 'input\\ogs201101_12_det.ele_ucorr', 'input\\ogs201201_12_det.ele_ucorr', 'input\\ogs201301_12_det.ele_ucorr']
1450
1444
Unique / total for 2-3: 0.996
['input\\ogs201401_12_det.ele_ucorr', 'input\\ogs201501_12_det.ele_ucorr', 'input\\ogs201601_12_det.ele_ucorr', 'input\\ogs201701_12_det.ele_ucorr']
1091
1049
Unique / total for 2-7: 0.962
['input\\ogs201901_12_det.ele_ucorr', 'input\\ogs202001_12_det.ele_ucorr', 'input\\ogs202101_12_det.ele_ucorr', 'input\\ogs202201_12_det.ele_ucorr', 'input\\ogs202301_12_det.ele_ucorr']
3104
3051
Unique / total for 2-3: 0.983
Number of detectio

<Figure size 640x480 with 0 Axes>